<a href="https://colab.research.google.com/github/teitow/kagglekeggla/blob/master/practice_probability_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**문제 4**

`Name` 특성만을 이용하여 다양한 경우에 대한 생존률을 계산하라.

---



힌트: 아래 두 사이트의 내용을 참고한다.

- [Titanic using Name only [0.81818]](https://www.kaggle.com/code/cdeotte/titanic-using-name-only-0-81818)
- [Notes of Titanic on Kaggle](https://typewind.github.io/2017/07/25/titanic/)

# 이름에 포함된 'Mr.', 'Mrs.', 'Miss', 'Master.' 등의 호칭을 이용한 생존률 계산

In [2]:
# 필요한 라이브러리 임포트
import pandas as pd # 데이터 조작 및 분석
import re # 정규 표현식 (str.extract에서 내부적으로 사용됨)
# Google Colab 환경에서 Google Drive 연동을 위한 라이브러리
from google.colab import drive

# --- Google Drive 마운트 ---
# '/content/drive' 경로에 사용자의 Google Drive를 연결(마운트)합니다.
# 이 코드 셀을 실행하면 Google 계정 인증 및 권한 부여 요청이 팝업 또는 링크로 나타납니다.
try:
    drive.mount('/content/drive')
    # 마운트 성공 시 메시지 출력
    print("Google Drive 마운트가 성공적으로 완료되었습니다!")
except Exception as e:
    # 마운트 과정에서 오류 발생 시 메시지 출력 후 코드 실행 중단
    print(f"Google Drive 마운트 중 오류 발생: {e}")
    exit() # 프로그램 종료

# --- 파일 경로 설정 ---
# Google Drive 내의 파일 경로를 정확하게 지정합니다.
# '내 드라이브'는 '/content/drive/MyDrive/' 경로에 해당합니다.
# 사용자가 명시한 경로: '내 드라이브/kaggle/'
base_drive_path = '/content/drive/MyDrive/kaggle/' # Google Drive 내의 kaggle 폴더 경로
train_file_path = base_drive_path + 'train.csv' # train.csv 파일의 전체 경로
test_file_path = base_drive_path + 'test.csv'   # test.csv 파일의 전체 경로

# --- 데이터 로드 ---
# 타이타닉 훈련 데이터셋 로드 시도 (생존율 분석에 사용)
try:
    # 지정된 Google Drive 경로에서 'train.csv' 파일을 pandas DataFrame으로 읽어옴
    df_train = pd.read_csv(train_file_path)
    # 훈련 데이터 로드 성공 메시지 출력, 파일 경로 포함
    print(f"훈련 데이터셋 '{train_file_path}'을(를) 성공적으로 로드했습니다.")
# 파일을 찾을 수 없을 경우 예외 처리
except FileNotFoundError:
    # 오류 메시지 출력 (경로 및 파일 존재 여부 확인 안내) 후 프로그램 종료
    print(f"오류: 훈련 데이터 파일 '{train_file_path}'을(를) 찾을 수 없습니다. Google Drive 경로와 파일명을 다시 확인해주세요.")
    exit() # 프로그램 종료
# 파일 읽기 중 발생할 수 있는 다른 예외 처리 (예: 권한 문제, 파일 형식 오류 등)
except Exception as e:
    print(f"훈련 데이터 파일 '{train_file_path}' 로드 중 오류 발생: {e}")
    exit() # 프로그램 종료

# 타이타닉 테스트 데이터셋 로드 시도 (참고용으로 로드, 생존율 분석에는 직접 사용 안 함)
try:
    # 지정된 Google Drive 경로에서 'test.csv' 파일을 pandas DataFrame으로 읽어옴
    df_test = pd.read_csv(test_file_path)
    # 테스트 데이터 로드 성공 메시지 출력, 파일 경로 포함
    print(f"테스트 데이터셋 '{test_file_path}'을(를) 성공적으로 로드했습니다.")
    # 주석: 이 스크립트는 train.csv의 'Survived' 컬럼을 이용한 생존율 분석이 주 목적입니다.
    # test.csv에는 'Survived' 컬럼이 없으므로, 로드된 df_test는 이 분석에 직접 사용되지 않습니다.
    # 만약 모델 예측 등을 수행한다면 df_test에도 동일한 전처리(예: Title 추출)를 적용해야 합니다.
# 파일을 찾을 수 없을 경우 예외 처리
except FileNotFoundError:
    # 경고 메시지만 출력하고 계속 진행 (훈련 데이터 분석은 가능)
    print(f"경고: 테스트 데이터 파일 '{test_file_path}'을(를) 찾을 수 없습니다. 테스트 데이터 없이 훈련 데이터 분석만 진행합니다.")
    # df_test 변수를 None으로 설정하여 이후 코드에서 참조 시 오류 방지
    df_test = None
# 파일 읽기 중 발생할 수 있는 다른 예외 처리
except Exception as e:
    print(f"테스트 데이터 파일 '{test_file_path}' 로드 중 오류 발생: {e}")
    df_test = None # 오류 발생 시 df_test를 None으로 설정

# --- 호칭(Title) 추출 (훈련 데이터 대상) ---
# df_train의 'Name' 컬럼에서 정규 표현식을 이용해 호칭(Title)을 추출하여 새로운 'Title' 컬럼 생성
# ' ([A-Za-z]+)\.' 패턴: 공백 문자 뒤에 오고 점(.)으로 끝나는 하나 이상의 알파벳 문자열 그룹을 찾음
df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# --- 호칭(Title) 정제 및 그룹화 (훈련 데이터 대상) ---
# 프랑스식 여성 호칭 'Mlle'(마드모아젤)와 'Ms'를 일반적인 'Miss'로 통일
df_train['Title'] = df_train['Title'].replace(['Mlle', 'Ms'], 'Miss')
# 프랑스식 기혼 여성 호칭 'Mme'(마담)를 일반적인 'Mrs'로 통일
df_train['Title'] = df_train['Title'].replace(['Mme'], 'Mrs')
# 수가 매우 적은 귀족, 전문직, 군인, 성직자 등의 호칭들을 'Rare'라는 하나의 범주로 통합
# 통합 대상: 'Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'
df_train['Title'] = df_train['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

# --- 호칭별 생존율 계산 (훈련 데이터 사용) ---
# 'Title' 컬럼 값을 기준으로 훈련 데이터(df_train)를 그룹화(groupby)
# 각 'Title' 그룹에 대해 'Survived' 컬럼의 평균(mean)과 데이터 개수(count)를 계산(agg)
#   - 평균(mean): 해당 그룹의 생존율 (Survived 값이 0 또는 1이므로 평균이 곧 생존 비율)
#   - 개수(count): 해당 그룹에 속한 총 승객 수
survival_rate_by_title = df_train.groupby('Title')['Survived'].agg(['mean', 'count'])

# 계산 결과 DataFrame의 컬럼 이름을 좀 더 명확하게 변경
# 'mean' 컬럼 -> 'Survival Rate' (생존율)
# 'count' 컬럼 -> 'Count' (승객 수)
# inplace=True 옵션: 원본 DataFrame(survival_rate_by_title)을 직접 수정
survival_rate_by_title.rename(columns={'mean': 'Survival Rate', 'count': 'Count'}, inplace=True)

# 계산된 'Survival Rate'(생존율)을 기준으로 결과를 내림차순(높은 순서부터) 정렬
# ascending=False: 내림차순 정렬 의미
survival_rate_by_title = survival_rate_by_title.sort_values(by='Survival Rate', ascending=False)

# --- 결과 출력 ---
# 분석 결과 제목을 출력
print("\n--- 이름(Name) 특성 기반 호칭(Title)별 생존율 분석 결과 (훈련 데이터 기준) ---")
# 최종 계산된 호칭별 생존율과 해당 그룹의 승객 수를 담은 DataFrame을 출력
print(survival_rate_by_title)

# --- 추가 설명 ---
# 각 호칭 그룹이 일반적으로 어떤 승객 유형을 나타내는지 부가 설명을 출력
print("\n--- 각 호칭의 일반적인 의미 ---")
print(" - Mrs: 기혼 여성 (Married Woman)")
print(" - Miss: 미혼 여성 (Unmarried Woman, 주로 젊거나 소녀)")
print(" - Master: 어린 남성 (Young Boy)")
print(" - Rare: 귀족, 전문직, 군인, 성직자 등 수가 적은 특정 그룹 (Rare Titles)")
print(" - Mr: 성인 남성 (Adult Man)")

# 분석 결과의 의미와 요약을 설명
print("\n--- 분석 요약 ---")
print("이 분석은 Google Drive에 저장된 훈련 데이터(train.csv)를 기반으로 진행되었습니다.")
print("승객의 이름에 포함된 호칭(Title) 정보는 성별, 나이, 사회적 지위 등 생존에 영향을 미치는")
print("다른 특성들과 강한 상관관계를 가집니다. 이를 활용하여 호칭 그룹별 생존율을 계산했습니다.")
print("결과적으로 여성('Mrs', 'Miss')과 어린이('Master') 그룹의 생존율이 높고,")
print("성인 남성('Mr') 그룹의 생존율이 낮게 나타나는 타이타닉 데이터의 특징을 확인할 수 있습니다.")
print("(참고: 테스트 데이터(test.csv)도 함께 로드되었으나, 'Survived' 컬럼이 없기 때문에")
print("이 생존율 계산 분석에는 직접적으로 사용되지 않았습니다.)")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 마운트가 성공적으로 완료되었습니다!
훈련 데이터셋 '/content/drive/MyDrive/kaggle/train.csv'을(를) 성공적으로 로드했습니다.
테스트 데이터셋 '/content/drive/MyDrive/kaggle/test.csv'을(를) 성공적으로 로드했습니다.

--- 이름(Name) 특성 기반 호칭(Title)별 생존율 분석 결과 (훈련 데이터 기준) ---
        Survival Rate  Count
Title                       
Mrs          0.793651    126
Miss         0.702703    185
Master       0.575000     40
Rare         0.347826     23
Mr           0.156673    517

--- 각 호칭의 일반적인 의미 ---
 - Mrs: 기혼 여성 (Married Woman)
 - Miss: 미혼 여성 (Unmarried Woman, 주로 젊거나 소녀)
 - Master: 어린 남성 (Young Boy)
 - Rare: 귀족, 전문직, 군인, 성직자 등 수가 적은 특정 그룹 (Rare Titles)
 - Mr: 성인 남성 (Adult Man)

--- 분석 요약 ---
이 분석은 Google Drive에 저장된 훈련 데이터(train.csv)를 기반으로 진행되었습니다.
승객의 이름에 포함된 호칭(Title) 정보는 성별, 나이, 사회적 지위 등 생존에 영향을 미치는
다른 특성들과 강한 상관관계를 가집니다. 이를 활용하여 호칭 그룹별 생존율을 계산했습니다.
결과적으로 여성('Mrs', 'Miss')과 어린이('Master') 그룹의 생존율이 높고,
성인 남성('

# 성(Last Name)을 이용한 가족 단위 추정

In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
from google.colab import drive

# --- Google Drive 마운트 및 데이터 로드 (이전 코드와 동일) ---
try:
    drive.mount('/content/drive', force_remount=True) # 필요시 재마운트
    print("Google Drive 마운트 성공!")
except Exception as e:
    print(f"Google Drive 마운트 실패: {e}")
    exit()

base_drive_path = '/content/drive/MyDrive/kaggle/'
train_file_path = base_drive_path + 'train.csv'

try:
    df_train = pd.read_csv(train_file_path)
    print(f"훈련 데이터셋 '{train_file_path}' 로드 성공.")
except Exception as e:
    print(f"훈련 데이터 파일 로드 중 오류: {e}")
    exit()
# -----------------------------------------------------------

print("\n--- 방법 1: 성(Last Name) 기반 가족 크기 추정 및 생존율 분석 ---")

# 'Name' 컬럼에서 성(Last Name) 추출 (보통 쉼표 앞부분)
# 쉼표(',')를 기준으로 문자열을 나누고 첫 번째 요소(인덱스 0)를 가져옴
df_train['LastName'] = df_train['Name'].str.split(',').str[0]

# 각 승객과 동일한 성(LastName)을 가진 사람의 수를 계산하여 'FamilySize' 컬럼 생성
# groupby('LastName'): 같은 성을 가진 그룹 생성
# transform('count'): 각 그룹의 크기(동일 성을 가진 사람 수)를 계산하여 원래 DataFrame의 각 행에 매핑
df_train['FamilySize'] = df_train.groupby('LastName')['LastName'].transform('count')

# 추정된 'FamilySize'별 생존율 계산
# 'FamilySize'로 그룹화하고 'Survived'의 평균(생존율)과 그룹 크기(count) 계산
survival_by_family_size = df_train.groupby('FamilySize')['Survived'].agg(['mean', 'count'])

# 컬럼 이름 변경 (가독성)
survival_by_family_size.rename(columns={'mean': 'Survival Rate', 'count': 'Passenger Count'}, inplace=True)

# 결과 출력 (가족 크기 오름차순으로 정렬된 상태)
print(survival_by_family_size)

# 결과 해석 예시
print("\n[해석 예시]")
print(" - FamilySize가 1인 경우 (같은 성을 가진 사람이 본인뿐인 경우)의 생존율을 볼 수 있습니다.")
print(" - FamilySize가 2, 3, 4 등 증가함에 따라 생존율이 어떻게 변하는지 추세를 파악할 수 있습니다.")
print(" - (주의) 이 FamilySize는 성(Last Name)만을 기준으로 추정한 값이므로 실제 가족 구성과 다를 수 있습니다.")

Mounted at /content/drive
Google Drive 마운트 성공!
훈련 데이터셋 '/content/drive/MyDrive/kaggle/train.csv' 로드 성공.

--- 방법 1: 성(Last Name) 기반 가족 크기 추정 및 생존율 분석 ---
            Survival Rate  Passenger Count
FamilySize                                
1                0.359551              534
2                0.524096              166
3                0.357143               84
4                0.428571               56
5                0.000000                5
6                0.233333               30
7                0.000000                7
9                0.222222                9

[해석 예시]
 - FamilySize가 1인 경우 (같은 성을 가진 사람이 본인뿐인 경우)의 생존율을 볼 수 있습니다.
 - FamilySize가 2, 3, 4 등 증가함에 따라 생존율이 어떻게 변하는지 추세를 파악할 수 있습니다.
 - (주의) 이 FamilySize는 성(Last Name)만을 기준으로 추정한 값이므로 실제 가족 구성과 다를 수 있습니다.


# 이름 길이(Name Length) 분석

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
# import numpy as np # 필요시 사용 (예: 구간 나눌 때)
from google.colab import drive

# --- Google Drive 마운트 및 데이터 로드 (이전 코드와 동일) ---
try:
    # 이미 마운트 되었다면 재실행 시 바로 성공 메시지가 나올 수 있음
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive 마운트 성공!")
except Exception as e:
    print(f"Google Drive 마운트 실패: {e}")
    exit()

base_drive_path = '/content/drive/MyDrive/kaggle/'
train_file_path = base_drive_path + 'train.csv'

try:
    df_train = pd.read_csv(train_file_path)
    print(f"훈련 데이터셋 '{train_file_path}' 로드 성공.")
except Exception as e:
    print(f"훈련 데이터 파일 로드 중 오류: {e}")
    exit()
# -----------------------------------------------------------

print("\n--- 방법 2: 이름 길이(Name Length)와 생존율 분석 ---")

# 'Name' 컬럼의 각 이름 문자열 길이를 계산하여 'NameLength' 컬럼 생성
df_train['NameLength'] = df_train['Name'].str.len()

# 이름 길이를 구간(bin)으로 나누어 분석 (너무 세분화되는 것을 방지)
# 예: 15자 이하, 16-25자, 26-35자, 36-45자, 46자 이상으로 구간 설정
# bins: 구간 경계값 리스트
# labels: 각 구간의 이름
bins = [0, 15, 25, 35, 45, df_train['NameLength'].max()] # 0부터 최대 길이까지 구간 설정
labels = ['0-15', '16-25', '26-35', '36-45', '46+']
# pd.cut 함수를 사용하여 'NameLength'를 기반으로 'NameLengthBin' 컬럼 생성
df_train['NameLengthBin'] = pd.cut(df_train['NameLength'], bins=bins, labels=labels, right=True) # right=True: 각 구간의 오른쪽 경계를 포함

# 이름 길이 구간('NameLengthBin')별 생존율 계산
# 'NameLengthBin'으로 그룹화하고 'Survived'의 평균(생존율)과 그룹 크기(count) 계산
survival_by_name_length = df_train.groupby('NameLengthBin')['Survived'].agg(['mean', 'count'])

# 컬럼 이름 변경
survival_by_name_length.rename(columns={'mean': 'Survival Rate', 'count': 'Passenger Count'}, inplace=True)

# 결과 출력
print(survival_by_name_length)

# 결과 해석 예시
print("\n[해석 예시]")
print(" - 이름 길이 구간별로 생존율에 차이가 있는지 확인해 볼 수 있습니다.")
print(" - 예를 들어, 특정 길이 구간의 승객들이 다른 구간보다 높은/낮은 생존율을 보이는지 관찰합니다.")
print(" - (주의) 이름 길이와 생존율 간의 명확한 인과 관계를 찾기 어려울 수 있으며, 다른 요인(예: 호칭과 이름 길이가 우연히 연관)의 영향일 수 있습니다.")

Mounted at /content/drive
Google Drive 마운트 성공!
훈련 데이터셋 '/content/drive/MyDrive/kaggle/train.csv' 로드 성공.

--- 방법 2: 이름 길이(Name Length)와 생존율 분석 ---
               Survival Rate  Passenger Count
NameLengthBin                                
0-15                0.227273               22
16-25               0.277523              436
26-35               0.411371              299
36-45               0.620253               79
46+                 0.800000               55

[해석 예시]
 - 이름 길이 구간별로 생존율에 차이가 있는지 확인해 볼 수 있습니다.
 - 예를 들어, 특정 길이 구간의 승객들이 다른 구간보다 높은/낮은 생존율을 보이는지 관찰합니다.
 - (주의) 이름 길이와 생존율 간의 명확한 인과 관계를 찾기 어려울 수 있으며, 다른 요인(예: 호칭과 이름 길이가 우연히 연관)의 영향일 수 있습니다.


<ipython-input-4-7941fae80185>:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  survival_by_name_length = df_train.groupby('NameLengthBin')['Survived'].agg(['mean', 'count'])


정리 간단하게?

In [6]:
# -*- coding: utf-8 -*- # 파일 상단에 UTF-8 인코딩 명시 (한국어 주석 지원)

# 필요한 라이브러리 임포트
import pandas as pd # 데이터 분석 및 조작을 위한 pandas 라이브러리
import re # 정규 표현식(Regular Expression) 작업을 위한 re 라이브러리
# Google Colab 환경에서 Google Drive 연동을 위한 라이브러리
from google.colab import drive

# --- 1단계: Google Drive 마운트 및 데이터 로드 ---
# (이 부분은 환경 설정이므로 이전 코드와 동일하게 유지합니다)

# Google Drive를 '/content/drive' 경로에 마운트합니다.
try:
    drive.mount('/content/drive', force_remount=True) # 필요시 재마운트
    print("Google Drive 마운트 성공!")
except Exception as e:
    print(f"Google Drive 마운트 실패: {e}")
    exit()

# Google Drive 내 데이터 파일 경로를 설정합니다.
base_drive_path = '/content/drive/MyDrive/kaggle/'
train_file_path = base_drive_path + 'train.csv'

# 훈련 데이터셋(train.csv) 로드를 시도합니다.
try:
    df_train = pd.read_csv(train_file_path)
    print(f"훈련 데이터셋 '{train_file_path}' 로드 성공.")
except Exception as e:
    print(f"훈련 데이터 파일 로드 중 오류: {e}")
    exit()

# --- 2단계: 호칭(Prefix) 추출 및 숫자 변환 함수 정의 (사이트 코드 기반) ---

# << 제공해주신 'Notes of Titanic on Kaggle'의 코드 스니펫을 기반으로 함수 정의 >>
# 이 함수는 데이터셋을 입력받아 'Prefix' 컬럼을 추출하고 숫자로 변환하여 추가합니다.
def get_prefix(dataset):
    # 데이터셋에서 'Name' 컬럼을 가져옵니다 (사이트 코드의 'x_train["Name"]'에 해당).
    names = dataset["Name"]
    # 추출된 호칭(문자열)을 저장할 빈 리스트를 생성합니다 (사이트 코드와 동일).
    prefix = [] # 사이트 코드의 변수명 'prefix' 사용
    # 'Name' 컬럼의 각 이름에 대해 반복합니다 (사이트 코드와 동일).
    for name in names:
        # 정규표현식 ',(.*?)\.'을 사용하여 이름에서 호칭 부분을 추출합니다 (사이트 코드와 동일).
        # .group(1)으로 괄호 안의 매칭된 부분(호칭)을 가져옵니다 (예: ' Mr').
        # 사이트 코드에는 없지만, 혹시 모를 오류 방지를 위해 try-except 추가 (패턴 못 찾을 경우 대비)
        try:
            # re.search()를 사용하여 패턴 검색 및 그룹 추출
            extracted_prefix = re.search(',(.*?)\.', name).group(1)
        except AttributeError:
             # 패턴을 찾지 못한 경우 'Unknown' 또는 다른 값으로 처리 가능 (여기서는 일단 빈 문자열로)
            extracted_prefix = '' # 예외 발생 시 빈 문자열 추가
        # 추출된 호칭을 리스트에 추가합니다 (사이트 코드와 동일).
        prefix.append(extracted_prefix)

    # 추출된 호칭 리스트를 사용하여 데이터셋에 'Prefix' 컬럼을 추가합니다 (사이트 코드와 동일).
    # pd.Series를 이용해 리스트를 시리즈로 만들고 인덱스를 맞춰줍니다.
    dataset["Prefix"] = pd.Series(prefix, index=dataset.index)

    # 'Prefix' 컬럼(문자열)을 다시 가져와 Series 객체로 만듭니다 (사이트 코드와 동일한 구조).
    prefix_series = pd.Series(dataset["Prefix"]) # 변수명 변경하여 의미 명확화 (원래 코드: prefix = pd.Series(...))

    # .replace() 메소드를 사용하여 문자열 호칭을 숫자 값으로 변환합니다 (사이트 코드와 동일).
    # Mr=0, Woman=1, Child=2, Rare=4 규칙 적용. 호칭 앞 공백 포함 주의.
    prefix_series = prefix_series.replace(" Mr", 0) # ' Mr'를 숫자 0으로 변환
    prefix_series = prefix_series.replace([" Miss", " Mrs", " Ms", " Mme", " Mlle"], 1) # 여성 관련 호칭들을 숫자 1로 변환
    prefix_series = prefix_series.replace(" Master", 2) # ' Master'를 숫자 2로 변환
    # 희귀 호칭들을 리스트로 지정하여 숫자 4로 변환 (사이트 코드와 동일, Dona는 train 데이터에 없음)
    prefix_series = prefix_series.replace([' Rev', ' Dr', ' Col', ' Major', ' the Countess', ' Capt', ' Dona', 'Don', ' Sir', ' Lady', ' Jonkheer'], 4)

    # 최종 변환된 숫자 Series를 다시 데이터셋의 'Prefix' 컬럼에 저장합니다 (사이트 코드와 동일).
    dataset["Prefix"] = prefix_series
    # 'Prefix' 컬럼이 추가/수정된 데이터셋을 반환합니다 (사이트 코드와 동일).
    return dataset

# --- 3단계: 정의된 함수 적용 및 결과 확인 ---

# 위에서 정의한 get_prefix 함수를 훈련 데이터(df_train)에 적용합니다.
# 이 함수는 df_train을 직접 수정하고 반환합니다.
df_train_processed = get_prefix(df_train) # 원본 df_train을 변경할 수도 있지만, 결과 변수를 새로 받음

# 함수 적용 완료 및 결과 확인 메시지를 출력합니다.
print("\n'get_prefix' 함수 적용 완료 (사이트 코드 기반).")
# 변환된 'Prefix' 컬럼의 값 종류와 개수를 확인합니다. (0, 1, 2, 4 로 나와야 함)
print("\n변환된 숫자 Prefix 값 분포:")
print(df_train_processed['Prefix'].value_counts())

# --- 4단계: 숫자 그룹별 생존율 계산 ---
# (이 부분은 생존율 계산을 위해 필요한 표준 pandas 작업입니다)

# 숫자형 'Prefix' 컬럼(0, 1, 2, 4)을 기준으로 그룹화합니다.
# 각 그룹('Prefix' 값)별로 'Survived' 컬럼의 평균('mean', 생존율)과 개수('count')를 계산합니다.
survival_rate_by_prefix = df_train_processed.groupby('Prefix')['Survived'].agg(['mean', 'count'])

# 계산 결과 DataFrame의 컬럼 이름을 알아보기 쉽게 변경합니다.
survival_rate_by_prefix.rename(columns={'mean': 'Survival Rate', 'count': 'Passenger Count'}, inplace=True)

# --- 5단계: 최종 결과 출력 및 해석 ---

# 최종 분석 결과 제목을 출력합니다.
print("\n--- 'Name' 기반 숫자 호칭(Prefix) 그룹별 생존율 분석 결과 (사이트 코드 적용) ---")
# 계산된 그룹별 생존율 및 해당 그룹의 승객 수를 출력합니다.
print(survival_rate_by_prefix)

# 각 숫자 Prefix 값이 어떤 그룹을 나타내는지 설명하는 부분을 추가합니다 (발표 자료에 활용).
print("\n--- Prefix 숫자 그룹 의미 (사이트 설명 기반) ---")
print(" 0: 'Mr' (성인 남성)")
print(" 1: 'Miss', 'Mrs', 'Ms', 'Mme', 'Mlle' (여성)")
print(" 2: 'Master' (어린 남성)")
print(" 4: 'Rev', 'Dr', 'Col', 'Major', 'Countess', 'Capt', 'Don', 'Sir', 'Lady', 'Jonkheer' 등 (희귀 호칭)")

# 분석 요약 (발표 자료 참고용)
print("\n--- 분석 요약 (사이트 코드 중심 설명) ---")
print("1. 'Notes of Titanic on Kaggle' 사이트에서 제공된 코드 로직을 활용하여,")
print("   'Name' 컬럼에서 정규표현식으로 호칭('Prefix')을 추출했습니다.")
print("2. 추출된 호칭은 사이트의 설명에 따라 'Mr'(0), 'Woman'(1), 'Child'(2), 'Rare'(4)의")
print("   4개 그룹으로 분류되어 숫자 값으로 변환되었습니다.")
print("3. 이 숫자 그룹을 기준으로 생존율을 계산한 결과, 여성(1)과 어린이(2) 그룹의 생존율이 높고,")
print("   성인 남성(0) 그룹의 생존율이 낮음을 확인했습니다.")
print("4. 희귀 호칭(4) 그룹의 생존율도 비교적 높게 나타났습니다.")
print("5. 이는 해당 사이트에서 설명한 바와 같이, 이름 속 호칭 정보가 생존율과 관련된")
print("   중요한 정보(성별, 나이, 지위 등)를 담고 있음을 보여줍니다.")

Mounted at /content/drive
Google Drive 마운트 성공!
훈련 데이터셋 '/content/drive/MyDrive/kaggle/train.csv' 로드 성공.

'get_prefix' 함수 적용 완료 (사이트 코드 기반).

변환된 숫자 Prefix 값 분포:
Prefix
0       517
1       311
2        40
4        22
 Don      1
Name: count, dtype: int64

--- 'Name' 기반 숫자 호칭(Prefix) 그룹별 생존율 분석 결과 (사이트 코드 적용) ---
        Survival Rate  Passenger Count
Prefix                                
0            0.156673              517
1            0.739550              311
2            0.575000               40
4            0.363636               22
 Don         0.000000                1

--- Prefix 숫자 그룹 의미 (사이트 설명 기반) ---
 0: 'Mr' (성인 남성)
 1: 'Miss', 'Mrs', 'Ms', 'Mme', 'Mlle' (여성)
 2: 'Master' (어린 남성)
 4: 'Rev', 'Dr', 'Col', 'Major', 'Countess', 'Capt', 'Don', 'Sir', 'Lady', 'Jonkheer' 등 (희귀 호칭)

--- 분석 요약 (사이트 코드 중심 설명) ---
1. 'Notes of Titanic on Kaggle' 사이트에서 제공된 코드 로직을 활용하여,
   'Name' 컬럼에서 정규표현식으로 호칭('Prefix')을 추출했습니다.
2. 추출된 호칭은 사이트의 설명에 따라 'Mr'(0), 'Woman'(1), 'Child'(2), 'Rare'(4)의
  